In [1]:
import re
import requests

In [2]:
current = ""
fin = open("king.txt") # file downloaded from http://www.gutenberg.org/1/10/
for line in fin.readlines():
        current += line
print (len(current))
#print (current[:1000])
fin.close()
print (type(current))
#start here
#"*** START OF THIS PROJECT GUTENBERG EBOOK THE KING JAMES BIBLE ***"

#end here
#"End of the Project Gutenberg EBook of The King James Bible"

4353385
<class 'str'>


Now that I have managed to extract the text file into one long sting, it is time to clean it up and organize it into books, chapters, and verses.

First of all, I start by getting rid off the text at the beginning and the end of file that I don't need. That includes all the guternberg introductory infoamation at the beginning of the file and and the copyright and disclaimers text at the end.

In [3]:
#remove the informational text at the beginning 
currentB = current.split("*** START OF THIS PROJECT GUTENBERG EBOOK THE KING JAMES BIBLE ***", 1)[1]
print (len(currentB))

4352846


In [4]:
#remove the copyright and disclaimers text at the end
currentE = currentB.rsplit("End of the Project Gutenberg EBook of The King James Bible", 1)[0]
print (len(currentE))

4334126


To make sure that my techniques for stripping out the beginning and ending parts of the file have worked, I am going to print out the beginning and sentences of the current text, and make sure they look like what I want after comparing them to the original file.

In [5]:
#print (currentE[80:200])

In [6]:
#print (currentE[len(currentE)-300:len(currentE)])

That looks right, the next step now is to find a good splitting char or combination of chars, so that organizing the current text into books, chapters, and verses will be a straightforward process.

In [7]:
currentT = currentE.split("***")
print (len(currentT)) #this is correct, as we have two testaments in the text file divided by the *** characters.

2


Now starting with the old testament, I am about to start sorting the whole text by verse, chapter, and book. By the end of the process, I should have a total of 66 books 39 from the old testament and 27 books from the new testament.

In [8]:
newTest = currentT[1]
oldTest = currentT[0]
#print newTest[:20]
#print (oldTest[:100])

In [9]:
oldT = oldTest.split("\nANOTHER BOOK STARTS HERE")
newT = newTest.split("\nANOTHER BOOK STARTS HERE")
print (len(oldT))
print (len(newT))
# split on "\nANOTHER BOOK STARTS HERE"
# then build a tree-like structure from book, to chapter, then verse that can used to construct a url

39
27


In [10]:
oldTBooks = ["Genesis","Exodus","Leviticus","Numbers","Deuteronomy","Joshua","Judges","Ruth","1 Samuel","2 Samuel",
           "1 Kings","2 Kings","1 Chronicles","2 Chronicles","Ezra","Nehemiah","Esther","Job","Psalms","Proverbs",
           "Ecclesiastes","Song of Solomon","Isaiah","Jeremiah","Lamentations","Ezekiel","Daniel","Hosea",
           "Joel","Amos","Obadiah","Jonah","Micah","Nahum","Habakkuk","Zephaniah","Haggai","Zechariah","Malachi"]

   

In [11]:
newTBooks = ["Matthew","Mark","Luke","John","Acts","Romans","1 Corinthians","2 Corinthians","Galatians","Ephesians",
             "Philippians","Colossians","1 Thessalonians","2 Thessalonians","1 Timothy","2 Timothy","Titus","Philemon",
             "Hebrews","James","1 Peter","2 Peter","1 John","2 John","3 John","Jude","Revelation"] 


In [12]:
booksNames = ["Genesis","Exodus","Leviticus","Numbers","Deuteronomy","Joshua","Judges","Ruth","1 Samuel","2 Samuel",
           "1 Kings","2 Kings","1 Chronicles","2 Chronicles","Ezra","Nehemiah","Esther","Job","Psalms","Proverbs",
           "Ecclesiastes","Song of Solomon","Isaiah","Jeremiah","Lamentations","Ezekiel","Daniel","Hosea",
           "Joel","Amos","Obadiah","Jonah","Micah","Nahum","Habakkuk","Zephaniah","Haggai","Zechariah","Malachi", 
            "Matthew","Mark","Luke","John","Acts","Romans","1 Corinthians","2 Corinthians","Galatians","Ephesians",
             "Philippians","Colossians","1 Thessalonians","2 Thessalonians","1 Timothy","2 Timothy","Titus",
            "Philemon","Hebrews","James","1 Peter","2 Peter","1 John","2 John","3 John","Jude","Revelation"]
booksAbbr = ["gen", "exo", "lev", "num", "deu", "jos", "jdg", "rut", "1sa", "2sa", "1ki", "2ki", "1ch", "2ch", "ezr", 
             "neh", "est", "job", "psa", "pro", "ecc", "sng", "isa", "jer", "lam", "ezk", "dan", "hos", "jol", "amo", 
             "oba", "jon", "mic", "nam", "hab", "zep", "hag", "zec", "mal", "mat", "mrk", "luk", "jhn", "act", "rom",
             "1co", "2co", "gal", "eph", "php", "col", "1th", "2th", "1ti", "2ti", "tit", "phm", "heb", "jas", "1pe",
             "2pe", "1jn", "2jn", "3jn", "jud", "rev"]

In [13]:
abbreviations = {}
for idx in range(len(booksNames)):
    abbreviations[booksNames[idx]] = booksAbbr[idx]

In [14]:
fullDict = {}
for i in range(len(oldTBooks)):
    fullDict[oldTBooks[i]] = oldT[i]
for i in range(len(newTBooks)):
    fullDict[newTBooks[i]] = newT[i]


In [19]:


versesDict = {}
for key, value in fullDict.items():
    verses = re.split(r'(\d+:\d+)', value)
    lst = []
    #print(len(verses))
    for k in range(1, len(verses), 2):
        
        newDict = {}
        newLst = []
        chapterVerseLst = verses[k].split(":")
        ch, ve = chapterVerseLst[0], chapterVerseLst[1]
        lst.append((ch, ve))
#         url = "https://www.bible.com/en-GB/bible/351/" + abbreviations[key] + "." + ch + "." + ve + ".bysb"
#         r = requests.get(url, auth=('user', 'pass'))
#         verseKinya = "Not Working"
#         if 'class="content">' in r.text:
#             verseKinya = r.text.rsplit('class="content">', 1)[1].split('</span></span>')[0]
#         else:
#             #print(verseKinya)
#             filename = key + " " + verses[k] + ".txt"
#             f = open(filename, 'w')
#             f.write(r.text)
#             f.close()
#         newLst.append(verses[k+1])
#         newLst.append(verseKinya)
#         newDict[verses[k]] = newLst
#         lst.append(newDict)
    versesDict[key] = lst

In [25]:
versesDict = {}
for key, value in fullDict.items():
    verses = re.split(r'(\d+:\d+)', value)
    ff = open(key+".txt", 'w')
    sss = ""
    #print(len(verses))
    for k in range(1, len(verses), 2):
        
        newDict = {}
        newLst = []
        chapterVerseLst = verses[k].split(":")
        sss += verses[k] + "\n"
        ch, ve = chapterVerseLst[0], chapterVerseLst[1]
        lst.append((ch, ve))
#         url = "https://www.bible.com/en-GB/bible/351/" + abbreviations[key] + "." + ch + "." + ve + ".bysb"
#         r = requests.get(url, auth=('user', 'pass'))
#         verseKinya = "Not Working"
#         if 'class="content">' in r.text:
#             verseKinya = r.text.rsplit('class="content">', 1)[1].split('</span></span>')[0]
#         else:
#             #print(verseKinya)
#             filename = key + " " + verses[k] + ".txt"
#             f = open(filename, 'w')
#             f.write(r.text)
#             f.close()
#         newLst.append(verses[k+1])
#         newLst.append(verseKinya)
#         newDict[verses[k]] = newLst
#         lst.append(newDict)
    ff.write(sss)
    ff.close()

In [195]:
#with open("translationData.txt", 'w') as f:
    #for key, value in versesDict.items():
        #f.write('%s:%s\n' % (key, value))
        
#to read it back the dictionnary constructed above
data = dict()
with open("translationData.txt") as raw_data:
    for item if raw_data:
        if ':' in item:
            key,value = item.split(':', 1)
            data[key]=value
        else:
            pass

In [187]:

kk = 0
for i in versesDict:
    kk += 1
print (kk)
# print (fullDict["Matthew"])
#print (fullDict["Matthew"][0]["1:1"])

66


In [190]:
print (versesDict["Acts"][0])

{'1:1': [' The former treatise have I made, O Theophilus, of all that Jesus\nbegan both to do and teach, ', 'Muri cya gitabo cya mbere nanditse ibyo Yesu yabanje gukora no kwigisha byose, ']}


Now that I have got the English King James bible all sorted out and organized by verse, chapter and book; I am going to use the requests http library to access the kinyarwanda translation from bible.com

In [194]:
# #engl2kinya = ""
# kinya2engl = ""
# for i in versesDict:
#     for k in versesDict[i]:
#         for key, value in k.items():
#             #engl2kinya += value[0]
#             #engl2kinya += " . |||"
#             #engl2kinya += value[1]
#             #engl2kinya += "\n"
            
#             kinya2engl += value[1]
#             kinya2engl += "|||"
#             kinya2engl += value[0]
#             #kinya2engl += "\n"

# #f = open("engl2kinya.txt", 'w')
# #f.write(engl2kinya)
# #f.close() 

# t = open("kinya2engl.txt", 'w')
# t.write(kinya2engl)
# t.close()  

In [ ]:
# r = requests.get('https://www.bible.com/en-GB/bible/351/est.8.9.bysb', auth=('user', 'pass'))
# print(r.status_code)                #longest verse in the bible, why not!
# print(r.headers['content-type'])
# print (r.encoding)
# print(r.json())                     #doesn't work content type is text/html

The following code block is my attempt trying to find the best way to splot the html page that I got back from the bible.com website. 

In [ ]:
# print (r.text[:100])
# print(len(r.text))
# verseKinya = r.text.rsplit('class="content">', 1)[1].split('</span></span>')[0]
# print(len(ghgh))
# print(ghgh)
#ghgh[1][:700]
#ghgh[0][len(ghgh)-30:]
#verseKiny = r.text.rsplit("</span></span><div id='read_chapter'>")[0].lsplit('<span class="content">')[1]
#print(verseKiny)

I use the below code to generate an example file so that I could examine what kind of data I was getting back after calling r.text on my requests object.

In [ ]:
# f = open('longestVerse.txt', 'w')
# f.write(r.text)
# f.close()